In [1]:
import sys
import os
import numpy as np
project_root = os.path.abspath('..')
sys.path.append(project_root)
from src.gdeeplearn.simpleDL2 import SimpleDL

In [2]:
#laod data
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
images, labels = (x_train[0:1000].reshape(1000,28*28) , y_train[0:1000])
one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):   
    one_hot_labels[i][l] = 1
labels = one_hot_labels
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

2025-09-06 08:52:10.499057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757137930.520545    9797 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757137930.526494    9797 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757137930.541609    9797 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757137930.541631    9797 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757137930.541633    9797 computation_placer.cc:177] computation placer alr

In [3]:
avg_error , avg_accurcy = SimpleDL().GmodelV1(test_images,test_labels)

I: 0, Error: 0.42321, Correct: 0.74370 Test-Err:0.32121 Test-Acc:0.84900
I: 10, Error: 0.17138, Correct: 0.93300 Test-Err:0.18460 Test-Acc:0.93210
I: 20, Error: 0.16118, Correct: 0.93870 Test-Err:0.17341 Test-Acc:0.93720
I: 30, Error: 0.15597, Correct: 0.94150 Test-Err:0.16793 Test-Acc:0.94050
I: 40, Error: 0.15255, Correct: 0.94330 Test-Err:0.16473 Test-Acc:0.94260
I: 50, Error: 0.15003, Correct: 0.94430 Test-Err:0.16250 Test-Acc:0.94300
I: 60, Error: 0.14800, Correct: 0.94580 Test-Err:0.16064 Test-Acc:0.94450
I: 70, Error: 0.14644, Correct: 0.94670 Test-Err:0.15928 Test-Acc:0.94590
I: 80, Error: 0.14518, Correct: 0.94730 Test-Err:0.15814 Test-Acc:0.94630
I: 90, Error: 0.14413, Correct: 0.94750 Test-Err:0.15717 Test-Acc:0.94680
I: 100, Error: 0.14312, Correct: 0.94850 Test-Err:0.15649 Test-Acc:0.94780
I: 110, Error: 0.14227, Correct: 0.94870 Test-Err:0.15568 Test-Acc:0.94820
I: 120, Error: 0.14159, Correct: 0.94910 Test-Err:0.15509 Test-Acc:0.94840
I: 130, Error: 0.14096, Correct: 0.9

In [9]:
import torch
import torch.nn as nn
import numpy as np
from typing import Tuple
from torch.utils.data import TensorDataset, DataLoader

class PyTorchNeuralNetwork(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_labels: int, epochs: int = 300, alpha: float = 0.01, batch_size: int = 32):
        super(PyTorchNeuralNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_labels = num_labels
        self.epochs = epochs
        self.alpha = alpha
        self.batch_size = batch_size
        
        # Define layers
        self.layer_0_1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer_1_2 = nn.Linear(hidden_size, num_labels)
        
        # Initialize weights similar to your implementation (uniform [-0.1, 0.1])
        nn.init.uniform_(self.layer_0_1.weight, -0.1, 0.1)
        nn.init.uniform_(self.layer_1_2.weight, -0.1, 0.1)
        nn.init.uniform_(self.layer_0_1.bias, -0.1, 0.1)
        nn.init.uniform_(self.layer_1_2.bias, -0.1, 0.1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layer_0_1(x)
        x = self.relu(x)
        x = self.layer_1_2(x)
        return x

    def train_model(self, input_data: np.ndarray, labels: np.ndarray, test_images: np.ndarray = None, test_labels: np.ndarray = None) -> Tuple[float, float]:
        """Trains a neural network model with one hidden layer using gradient descent.
        
        Args:
            input_data (np.ndarray): Input data with shape (n_samples, n_features).
            labels (np.ndarray): Target labels with shape (n_samples, n_labels).
            test_images (np.ndarray, optional): Test input data with shape (n_test_samples, n_features). Defaults to input_data.
            test_labels (np.ndarray, optional): Test target labels with shape (n_test_samples, n_labels). Defaults to labels.
        
        Returns:
            Tuple[float, float]: Average training error and accuracy over the dataset.
        
        Notes:
            Uses PyTorch with mini-batching for training with ReLU activation and MSE loss.
            Evaluates on the test set every 10 epochs or on the last epoch.
            Results are printed to stdout.
        """
        # Convert numpy arrays to PyTorch tensors
        input_data = torch.from_numpy(input_data).float()
        labels = torch.from_numpy(labels).float()
        
        # Use input_data and labels as test set if test data not provided
        if test_images is None or test_labels is None:
            test_images = input_data
            test_labels = labels
        else:
            test_images = torch.from_numpy(test_images).float()
            test_labels = torch.from_numpy(test_labels).float()
        
        # Create DataLoaders for batching
        train_dataset = TensorDataset(input_data, labels)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_dataset = TensorDataset(test_images, test_labels)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
        
        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=self.alpha)
        
        total_error, correct_count = 0.0, 0
        for j in range(self.epochs):
            self.train()  # Set model to training mode
            error, correct_cnt = 0.0, 0
            for inputs, targets in train_loader:
                # Forward pass
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                error += loss.item() * len(inputs)
                correct_cnt += int(torch.sum(torch.argmax(outputs, dim=1) == torch.argmax(targets, dim=1)))
                
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            error /= len(input_data)
            correct_cnt /= len(input_data)
            total_error += error
            correct_count += correct_cnt
            print(f"\rI: {j}, Error: {error:.5f}, Correct: {correct_cnt:.5f}", end='')

            # Test evaluation every 10 epochs or on the last epoch
            if j % 10 == 0 or j == self.epochs - 1:
                self.eval()  # Set model to evaluation mode
                test_error, test_correct = 0.0, 0
                with torch.no_grad():
                    for test_inputs, test_targets in test_loader:
                        test_outputs = self(test_inputs)
                        test_error += criterion(test_outputs, test_targets).item() * len(test_inputs)
                        test_correct += int(torch.sum(torch.argmax(test_outputs, dim=1) == torch.argmax(test_targets, dim=1)))
                test_error /= len(test_images)
                test_accuracy = test_correct / len(test_images)
                print(f" Test-Err:{test_error:.5f} Test-Acc:{test_accuracy:.5f} Correct:{test_correct}")
                self.train()  # Set model back to training mode

        avg_error = total_error / self.epochs
        avg_accuracy = correct_count / self.epochs
        return avg_error, avg_accuracy



In [10]:
model = PyTorchNeuralNetwork(input_size=784, hidden_size=10, num_labels=10, epochs=300, alpha=0.01, batch_size=32)
avg_error, avg_accuracy = model.train_model(test_images, test_labels)

I: 0, Error: 0.09382, Correct: 0.19320 Test-Err:0.08774 Test-Acc:0.24120 Correct:2412
I: 10, Error: 0.06164, Correct: 0.60840 Test-Err:0.06082 Test-Acc:0.62580 Correct:6258
I: 20, Error: 0.05143, Correct: 0.72620 Test-Err:0.05104 Test-Acc:0.72920 Correct:7292
I: 30, Error: 0.04467, Correct: 0.78160 Test-Err:0.04433 Test-Acc:0.78280 Correct:7828
I: 40, Error: 0.03931, Correct: 0.81550 Test-Err:0.03907 Test-Acc:0.81470 Correct:8147
I: 50, Error: 0.03558, Correct: 0.83600 Test-Err:0.03539 Test-Acc:0.83830 Correct:8383
I: 60, Error: 0.03287, Correct: 0.85520 Test-Err:0.03272 Test-Acc:0.85690 Correct:8569
I: 70, Error: 0.03092, Correct: 0.86790 Test-Err:0.03081 Test-Acc:0.86770 Correct:8677
I: 80, Error: 0.02951, Correct: 0.87300 Test-Err:0.02942 Test-Acc:0.87400 Correct:8740
I: 90, Error: 0.02846, Correct: 0.87720 Test-Err:0.02839 Test-Acc:0.87830 Correct:8783
I: 100, Error: 0.02763, Correct: 0.88030 Test-Err:0.02756 Test-Acc:0.88140 Correct:8814
I: 110, Error: 0.02692, Correct: 0.88380 Te